## Preprocessing

### Imports

Here we import the data, and do the basic initial transformations like creating the unix time column

In [2]:
import numpy as np
import pandas as pd

np.random.seed(42)

import time
# import datetime
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [3]:
# helper functions

def data_split(df):
    """returns 10% of the data"""
    return df[: int((len(df)/10))]


def UnixTime(df, col="timestamp", newcol="Unix"):
    """Adds a new column to the dataframe containing the UNIX time of the timestamp"""
    cop = df.copy()
    cop[col] = pd.to_datetime(cop[col], dayfirst=True)
    unixTransform = lambda x: time.mktime(x.timetuple())
    df[newcol] = cop[col].apply(unixTransform)

In [4]:
path = r"D:/University/Year 2/Q3/DBL/Data/"
train = pd.read_csv(f'{path}BPI Challenge 2017/BPI_Challenge_2017-training.csv')
test = pd.read_csv(f'{path}BPI Challenge 2017/BPI_Challenge_2017-test.csv')


data = pd.concat([train, test])

The column and Unix time cell are slow, might need optimization

In [5]:
# strip whitespace of column names and add extra time columns
data.columns = data.columns.str.strip()
data["timestamp"] = data["event time:timestamp"].copy()
data["event time:timestamp"] = pd.to_datetime(data["event time:timestamp"], dayfirst=True)
data['time of day'] = data["timestamp"].str.split(expand=True)[1]

In [6]:
data["weekday"] = data["event time:timestamp"].dt.day_name()

In [7]:
UnixTime(data)

In [8]:
data.sort_values(by=['Unix'], inplace=True)
data.head()

,eventID,case LoanGoal,case ApplicationType,case concept:name,case RequestedAmount,event Action,event org:resource,event concept:name,event EventOrigin,event EventID,event lifecycle:transition,event time:timestamp,timestamp,time of day,weekday,Unix
0,0,Existing loan takeover,New credit,Application_652823628,20000.000,Created,User_1,A_Create Application,Application,Application_652823628,complete,2016-01-01 10:51:15.304,01-01-2016 10:51:15.304,10:51:15.304,Friday,1451641875.000
1,1,Existing loan takeover,New credit,Application_652823628,20000.000,statechange,User_1,A_Submitted,Application,ApplState_1582051990,complete,2016-01-01 10:51:15.352,01-01-2016 10:51:15.352,10:51:15.352,Friday,1451641875.000
2,2,Existing loan takeover,New credit,Application_652823628,20000.000,Created,User_1,W_Handle leads,Workflow,Workitem_1298499574,schedule,2016-01-01 10:51:15.774,01-01-2016 10:51:15.774,10:51:15.774,Friday,1451641875.000
3,3,Existing loan takeover,New credit,Application_652823628,20000.000,Deleted,User_1,W_Handle leads,Workflow,Workitem_1673366067,withdraw,2016-01-01 10:52:36.392,01-01-2016 10:52:36.392,10:52:36.392,Friday,1451641956.000
4,4,Existing loan takeover,New credit,Application_652823628,20000.000,Created,User_1,W_Complete application,Workflow,Workitem_1493664571,schedule,2016-01-01 10:52:36.403,01-01-2016 10:52:36.403,10:52:36.403,Friday,1451641956.000


In [9]:
# drop useless columns
def dropper(df, lbls=["eventID", "event EventID", "timestamp"]):
    df.drop(labels=lbls, axis=1, inplace=True)

dropper(data)

In [10]:
data["Unix"] = data["Unix"].astype(int)

### Event/time adder

In [12]:
# event shifting
data['next_event'] = data.groupby('case concept:name')['event concept:name'].shift(-1)
data["prev_event"] = data.groupby('case concept:name')['event concept:name'].shift(1)

In [13]:
# time shifting
data["nextUnix"] = data.groupby('case concept:name')['Unix'].shift(-1)
data["prevUnix"] = data.groupby('case concept:name')['Unix'].shift(1)
data["nextTime"] = data.groupby('case concept:name')['event time:timestamp'].shift(-1)
data["prevTime"] = data.groupby('case concept:name')['event time:timestamp'].shift(1)

In [14]:
len(data[data["prevTime"].isna()])/len(data)

0.026207988741269617

In [15]:
len(data[data["prevTime"].isna()])

31509

In [16]:
len(data)

1202267

In [17]:
import math

# Adding time features
data['day'] = data['event time:timestamp'].dt.day;
data['month'] = data['event time:timestamp'].dt.month;
data['hour'] = data['event time:timestamp'].dt.hour;
data['day_of_week'] = data['event time:timestamp'].dt.weekday;

# Cyclical encoding
data["hour"] = 2 * math.pi * data["hour"] / data["hour"].max()
data["hour_cos"] = np.cos(data["hour"])
data["hour_sin"] = np.sin(data["hour"])
data["day_of_week"] = 2 * math.pi * data["day_of_week"] / data["day_of_week"].max()
data["day_of_week_cos"] = np.cos(data["day_of_week"])
data["day_of_week_sin"] = np.sin(data["day_of_week"])

In [18]:
data['day_of_week'].unique()

array([4.1887902 , 5.23598776, 6.28318531, 0.        , 1.04719755,
       2.0943951 , 3.14159265])

In [21]:
data.to_pickle(f"{path}Pickles/processed2017.pkl")